# Further Development

Part 1: Increasing number of aspects.

One aspect is extracted from a call of the prompt. Collect the results to a python dictionary object. Then condense the senetences and output the sentences as a JSON object.

In [1]:
GAME_ASPECTS = [
    'Gameplay', 
    'Narrative', 
    'Accessibility', 
    'Sound', 
    'Graphics & Art Design', 
    'Performance', 
    'Bug', 
    'Suggestion', 
    'Price', 
    'Overall'
]

# previously
# aspects = ['Gameplay', 'Sound', 'Graphics', 'Performance', 'Bug', 'Suggestion', 'Price', 'Overall']

In [2]:
import reviews._reviews02 as reviews02

In [3]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

# select a comment to test with
review_text = reviews02.sample_01

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents([review_text], metadatas=[{"source":"review_01"}])

embedding_func = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db = Chroma.from_documents(docs, embedding_func)

retriever = db.as_retriever(search_kwargs={"k": 5})

/root/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/miniforge3/envs/fyp-test-wsl-tm/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
from langchain_community.llms import Ollama

llm = Ollama(model='llama2:7b-chat-q4_K_M', temperature=0.2)        # set to a lower temperature for more consistent results

In [5]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_core.prompts import PromptTemplate

prompt_template = \
'''You are reading reviews of a game to understand the characteristics of the game. Use the following pieces of context to answer user's question. 

{summaries}

Question: {question}

If you don't know the answer, output only "NA". Do NOT try to make up an answer. Do NOT output other text.'''

my_question_template = \
'''Extract the the following aspect of the game from the reviews. Output a short 20 words description for each aspect. The aspect is: '''

aspects = GAME_ASPECTS
aspects_response = {k: '' for k in aspects}

for aspect in aspects:
    my_question = my_question_template + f'{aspect}'
    print(my_question)

    chain =  RetrievalQAWithSourcesChain.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={
            "prompt": PromptTemplate(
                template=prompt_template,
                input_variables=["summaries", "question"],
            )
        },
        return_source_documents=True,
    )

    response = chain.invoke(
        {
            'question': my_question
        }
    )

    print('\'\'\'')
    print(response['answer'])
    print('\'\'\'')
    aspects_response[aspect] = response['answer']
    print('-'*10)
    print('Sources')
    for doc in response['source_documents']:
        print(doc.page_content)
        print(doc.metadata)
        print('-'*10)

    print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Extract the the following aspect of the game from the reviews. Output a short 20 words description for each aspect. The aspect is: Gameplay


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1
Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


'''

Gameplay:

* NA - No information provided in the review about gameplay.
'''
----------
Sources
poorly optimized, runs between 25 - 35 fps on both low and ultra settings. you ' d think that if ultra was 30 - 35 then low should be 60, but no. even with max settings game still looked odd after disabling up - scaling. a $ 70 title should run at 50 - 60 fps on the lowest settings minimum. and trying to optimize the settings to get better frames i have dumped to many hours into it to get a refund. so here i will sit and wait till they fix the performance.
{'source': 'review_01'}
----------
--------------------
Extract the the following aspect of the game from the reviews. Output a short 20 words description for each aspect. The aspect is: Narrative
'''

Narrative: NA
'''
----------
Sources
poorly optimized, runs between 25 - 35 fps on both low and ultra settings. you ' d think that if ultra was 30 - 35 then low should be 60, but no. even with max settings game still looked odd after dis

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1
Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


...

'''

Sure! Here are the aspects of the game related to performance that can be extracted from the review:

1. Frame rate: Poorly optimized, runs between 25-35 FPS on both low and ultra settings.
2. Up-scaling: Disabling up-scaling improves the game's appearance, but it should not be necessary to do so.
3. Minimum frame rate: A $70 title should run at 50-60 FPS on the lowest settings minimum.
4. Optimization: Trying to optimize settings to improve frames resulted in many hours of playtime and no refund.
'''
----------
Sources
poorly optimized, runs between 25 - 35 fps on both low and ultra settings. you ' d think that if ultra was 30 - 35 then low should be 60, but no. even with max settings game still looked odd after disabling up - scaling. a $ 70 title should run at 50 - 60 fps on the lowest settings minimum. and trying to optimize the settings to get better frames i have dumped to many hours into it to get a refund. so here i will sit and wait till they fix the performance.
{'source

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1
Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


'''

Sure! Here are the aspects of the game and their corresponding 20-word descriptions based on the review:

1. Suggestion for optimization: NA
2. Expected FPS range: 50-60 fps (minimum)
3. Actual FPS range: 25-35 fps (both low and ultra settings)
'''
----------
Sources
poorly optimized, runs between 25 - 35 fps on both low and ultra settings. you ' d think that if ultra was 30 - 35 then low should be 60, but no. even with max settings game still looked odd after disabling up - scaling. a $ 70 title should run at 50 - 60 fps on the lowest settings minimum. and trying to optimize the settings to get better frames i have dumped to many hours into it to get a refund. so here i will sit and wait till they fix the performance.
{'source': 'review_01'}
----------
--------------------
Extract the the following aspect of the game from the reviews. Output a short 20 words description for each aspect. The aspect is: Price


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


'''

Here are the aspects of the game related to price based on the review provided:

Price: $70 (NA)
'''
----------
Sources
poorly optimized, runs between 25 - 35 fps on both low and ultra settings. you ' d think that if ultra was 30 - 35 then low should be 60, but no. even with max settings game still looked odd after disabling up - scaling. a $ 70 title should run at 50 - 60 fps on the lowest settings minimum. and trying to optimize the settings to get better frames i have dumped to many hours into it to get a refund. so here i will sit and wait till they fix the performance.
{'source': 'review_01'}
----------
--------------------
Extract the the following aspect of the game from the reviews. Output a short 20 words description for each aspect. The aspect is: Overall
'''

Overall: Poorly optimized game with inconsistent framerates.
'''
----------
Sources
poorly optimized, runs between 25 - 35 fps on both low and ultra settings. you ' d think that if ultra was 30 - 35 then low should

It's noted that fewer chunks will be produced in the in-memory vector database -> all aspects will have the same chunk to generate content.

Keep that in mind, it gives us opportunity to adjust the way of splitting the text for storing in the vector database.

---

Bonus: Multiple aspects at once, from the aspects above, without using RetrievalQAWithSourcesChain

In [6]:
from langchain_core.prompts import ChatPromptTemplate

system_template = \
'''You are reading reviews of a game to understand the characteristics of the game. Use the following pieces of context to answer user's question.
'''

summary_template = \
'''Extract the following aspects of the game from the reviews, and write a short 20 words description for each aspect. The aspects are: {aspects}. Do NOT output other text.

The context is wrapped by three consecutive apostrophes. The context is as follows:
\'\'\'
{context}
\'\'\'
'''

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", summary_template),
])

chain = chat_prompt | llm
response = chain.invoke({"context":str(aspects_response), "aspects":str(aspects)})

print(response)

Here are the 20-word descriptions for each aspect of the game based on the reviews provided:

['Gameplay']: NA - No information provided in the review about gameplay.

['Narrative']': NA - No information provided in the review about narrative.

['Accessibility']: NA - No information provided in the review about accessibility.

['Sound']: NA - No information provided in the review about sound.

['Graphics & Art Design']:

* Poorly optimized graphics and art design (20 words)
* Even with max settings, the game's visuals look odd after disabling upscaling (20 words)

['Performance']:

1. Frame rate: Poorly optimized, runs between 25-35 FPS on both low and ultra settings. (20 words)
2. Up-scaling: Disabling up-scaling improves the game's appearance, but it should not be necessary to do so. (20 words)
3. Minimum frame rate: A $70 title should run at 50-60 FPS on the lowest settings minimum. (20 words)
4. Optimization: Trying to optimize settings to improve frames resulted in many hours of p

Observation

1. When more aspects are asked in the same sentence, while equal importance should be paid to each aspect, there is a random attention shift in prompting the model, which leads to ignorance of some aspects, or more than expected aspects

2. When prompting for more aspects (1 aspect at a time vs all aspects), more information can be provided in 1 aspect at a prompt, which is as expected due to the generative nature of the model. Optimization may exist in targeting with different length of content for more efficient token usage.